# DS Lab - Final Assignment: Robust Journey Planning #

## Load the data

In [1]:
#Importations.
import pandas as pd
import glob
from geopy import distance
import numpy as np

### Load the data published by the Open Data Platform Swiss Public Transport (https://opentransportdata.swiss).



In [2]:
#path on my computer of the files.
path = "2017-09-test/*.csv"

In [3]:
#useful list
data = []
#read each csv we had.
for fname in sorted(glob.glob(path)):
    d = pd.read_csv(fname, sep=';',low_memory=False)
    data.append(d)
    
#process the data from the csv files to a dataframe.
data = pd.concat(data)
data = pd.DataFrame(data)
data = data.reset_index()
data = data.drop(['index'], axis = 1)

In [4]:
#check the tail.
data.tail()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
1776718,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504294,Vuiteboeuf,14.09.2017 05:57,NaN,UNBEKANNT,14.09.2017 05:57,NaN,UNBEKANNT,False
1776719,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504295,Baulmes,14.09.2017 06:01,NaN,UNBEKANNT,14.09.2017 06:01,NaN,UNBEKANNT,False
1776720,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504296,Six-Fontaines,14.09.2017 06:05,NaN,UNBEKANNT,14.09.2017 06:05,NaN,UNBEKANNT,False
1776721,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504297,Trois-Villes,14.09.2017 06:12,NaN,UNBEKANNT,14.09.2017 06:12,NaN,UNBEKANNT,False
1776722,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504298,Ste-Croix,14.09.2017 06:21,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False


### Load the station list data.

In [5]:
#path.
station_path = "BFKOORD_GEO"

In [6]:
station_file  = open(station_path, "r")

In [7]:
BPUIC = []
long = []
lat = []
name = []

for line in station_file:
    l = line.split()
    BPUIC.append(l[0])
    long.append(l[1])
    lat.append(l[2])
    name.append(l[5])


In [8]:
d = {'BPUIC':BPUIC,
     'longitude' : long,
     'latitude' : lat,
     'station name': name}
station_df = pd.DataFrame(d)
station_df = station_df[['BPUIC','longitude','latitude']]
station_df['BPUIC'] = station_df['BPUIC'].astype(np.int64)

In [9]:
station_df.head()

,BPUIC,longitude,latitude
0,2,26.074412,44.446770
1,3,1.811446,50.901549
2,4,1.075329,51.284212
3,5,-3.543547,50.729172
4,7,9.733756,46.922368


In [10]:
zurich_coord = (47.407665036,8.540331172)
salamanca = (40.96, -5.50)

In [11]:
print(distance.distance(zurich_coord, salamanca).km)

1328.6915399458765


In [12]:
df = pd.merge(data,station_df,on='BPUIC',how='left')

In [13]:
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,longitude,latitude
0,13.09.2017,80:06____:17010:000,80:06____,DB,DB Regio AG,Zug,17010,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 05:45,13.09.2017 05:45:00,PROGNOSE,False,7.607284,47.568147
1,13.09.2017,80:06____:17012:000,80:06____,DB,DB Regio AG,Zug,17012,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 06:34,13.09.2017 06:34:00,PROGNOSE,False,7.607284,47.568147
2,13.09.2017,80:06____:17013:000,80:06____,DB,DB Regio AG,Zug,17013,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 06:25,13.09.2017 06:28:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147
3,13.09.2017,80:06____:17014:000,80:06____,DB,DB Regio AG,Zug,17014,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 09:48,13.09.2017 09:48:00,PROGNOSE,False,7.607284,47.568147
4,13.09.2017,80:06____:17015:000,80:06____,DB,DB Regio AG,Zug,17015,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 08:06,13.09.2017 08:11:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147


# Create Network

In [14]:
import networkx as nx
import datetime
from datetime import timezone
import math

In [78]:
# transform a date to an integer. Dates within the same 24 hours
# interval map to the same integer. the interval is specied by the
# origin argument. 
# example origin = 2017.09.13 2h00
# date_to_integer(2017.09.13 14h05) returns 0
# date_to_integer(2017.09.14 1h28) returns 0
# date_to_integer(2017.09.14 8h38) returns 1
def date_to_integer(row, origin=datetime.datetime(2017, 9, 13, 2, 0, 0)):
    # set reference time in seconds
    origin_seconds = origin.replace(tzinfo=timezone.utc).timestamp()
    # transform datetime into seconds    
    try:
        # transform datetime into seconds
        # row['scheduled_departure_time'].
        curr_datetime_seconds = row[0].replace(tzinfo=timezone.utc).timestamp()
    except ValueError:
        try:
            # row['scheduled_arrival_time']
            curr_datetime_seconds = row[1].replace(tzinfo=timezone.utc).timestamp()
        except ValueError:
            print('Time information not found !')
    # set target
    target = curr_datetime_seconds - origin_seconds
    # set offset
    offset = 60*60*24
    # find integer value to associate with row
    x = -2
    while not (x*offset <= target < (x+1)*offset):
        x += 1
    return x

In [79]:
# Renaming the columns of the dataframe
df.columns = ['trip_date', 'trip_id', 'operator_id', 'operator_abk', 
                'operator_name', 'product_id', 'line_id', 'line_text',
                'circulation_id', 'transport_text', 'additional_trip',
                'failed', 'stop_id', 'stop_name', 'scheduled_arrival_time',
                'actual_arrival_time', 'actual_arrival_time_status', 'scheduled_departure_time',
                'actual_departure_time',  'actual_departure_time_status', 'no_stop',
                'longitude', 'latitude']

In [80]:
# Convert columns 'trip_date', scheduled_arrival_time' and 'scheduled departure time' to datetime
df['trip_date'] = pd.to_datetime(df['trip_date'], format='%d.%m.%Y', errors='ignore')
df['scheduled_arrival_time'] = pd.to_datetime(df['scheduled_arrival_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['scheduled_departure_time'] = pd.to_datetime(df['scheduled_departure_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')
df['actual_departure_time'] = pd.to_datetime(df['actual_departure_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')

In [95]:
info_trips = pd.DataFrame()
visu_trips = pd.DataFrame()

# id_ = df['trip_id'].unique()[555]
for id_ in df['trip_id'].unique()[400:600]:
    temp = df.loc[df['trip_id'] == id_].copy()
    temp['date_id'] = temp[['scheduled_departure_time', 'scheduled_arrival_time']].apply(date_to_integer, axis=1)
    # We consider trips having at least 2 stops
    if len(temp) > 1:
        # For a given trip id sort arrival times at the stops
        temp = temp.sort_values(by=['line_id', 'date_id', 'scheduled_arrival_time'], axis='index', ascending=True, inplace=False, na_position='first')
        # Select only relevant columns in the dataframe
        temp = temp[['operator_name', 'product_id', 'line_id', 'date_id', 'scheduled_departure_time', 'actual_departure_time', 
                     'scheduled_arrival_time', 'actual_arrival_time', 'stop_name', 'longitude', 'latitude']]
        # Concatenate the result into a final DataFrame
        info_trips = pd.concat([info_trips, temp], axis=0, levels=['trip_id', 'product_id'], ignore_index=False)
        

        
        # get all trip dates for the current trip_id
        dates = info_trips['date_id'].unique()
        # get one available trip date as the schedule is assumed the same irrelevant
        # of the trip date
        
        d = dates[0] if not dates[0] == float('NaN') else dates[1]
        # Create a visualization DataFrame showing bus stops, 
        # departure time and arrival time for each trip id        
        visu = temp[temp['date_id'] == d].copy()
        # define sequence of stops for the trip
        visu['stop_list'] = ' | '.join(visu['stop_name'])
        # Departure time at the start of the trip
        visu['scheduled_terminus_departure_time'] = visu['scheduled_departure_time'].iloc[0]
        # Actual departure time at the start of the trip
        visu['actual_terminus_departure_time'] = visu['actual_departure_time'].iloc[0]
        # Arrival time at the end of the trip
        visu['scheduled_terminus_arrival_time'] = visu['scheduled_arrival_time'].iloc[len(visu)-1]
        # Actual arrival time at the end of the trip
        visu['actual_terminus_arrival_time'] = visu['actual_arrival_time'].iloc[len(visu)-1]
        
        #visu = visu.reset_index(['operator_name', 'product_id', 'line_id'])
        # Select only relevant columns in the dataframe
        visu = visu[['operator_name', 'product_id', 'line_id', 'date_id', 'stop_list', 'scheduled_terminus_departure_time', 
                     'scheduled_terminus_arrival_time', 'actual_terminus_departure_time', 
                     'actual_terminus_arrival_time']].iloc[:1]
        # Concatenate the result into a final DataFrame
        visu_trips = pd.concat([visu_trips, visu], axis=0, ignore_index=False)

In [43]:
info_trips = pd.DataFrame()
visu_trips = pd.DataFrame()

# id_ = df['trip_id'].unique()[555]
for id_ in df['trip_id'].unique()[400:600]:
    temp = df.loc[df['trip_id'] == id_]
    # We consider trips having at least 2 stops
    if len(temp) > 1:
        # For a given trip id sort arrival times at the stops
        temp = temp.sort_values(by=['line_id', 'scheduled_arrival_time'], axis='index', ascending=True, inplace=False, na_position='first')
        # Select only relevant columns in the dataframe
        temp = temp[['operator_name', 'product_id', 'line_id', 'scheduled_departure_time', 'actual_departure_time', 
                     'scheduled_arrival_time', 'actual_arrival_time', 'stop_name', 'longitude', 'latitude']]
        # Concatenate the result into a final DataFrame
        info_trips = pd.concat([info_trips, temp], axis=0, levels=['line_id', 'product_id'], ignore_index=False)

        # Create a visualization DataFrame showing bus stops, 
        # departure time and arrival time for each trip id
        visu = temp.copy()
        visu['stop_list'] = ' | '.join(visu['stop_name'])
        # Departure time at the start of the trip
        visu['scheduled_terminus_departure_time'] = visu['scheduled_departure_time'].iloc[0]
        # Actual departure time at the start of the trip
        visu['actual_terminus_departure_time'] = visu['actual_departure_time'].iloc[0]
        # Arrival time at the end of the trip
        visu['scheduled_terminus_arrival_time'] = visu['scheduled_arrival_time'].iloc[len(visu)-1]
        # Actual arrival time at the end of the trip
        visu['actual_terminus_arrival_time'] = visu['actual_arrival_time'].iloc[len(visu)-1]
        
        #visu = visu.reset_index(['operator_name', 'product_id', 'line_id'])
        # Select only relevant columns in the dataframe
        visu = visu[['operator_name', 'product_id', 'line_id', 'stop_list', 'scheduled_terminus_departure_time', 
                     'scheduled_terminus_arrival_time', 'actual_terminus_departure_time', 
                     'actual_terminus_arrival_time']].iloc[:1]
        # Concatenate the result into a final DataFrame
        visu_trips = pd.concat([visu_trips, visu], axis=0, ignore_index=False)

In [96]:
# Add date_id column to better order days. (Here a day is considered to start at 2am and finish at 1.59am)
# info_trips['date_id'] = info_trips[['scheduled_departure_time', 'scheduled_arrival_time']].apply(date_to_integer, axis=1)
info_trips = info_trips.set_index(['operator_name', 'line_id', 'date_id', 'product_id']).sort_index(axis=0, level=['line_id'])
info_trips = info_trips[['stop_name', 'longitude', 'latitude', 'scheduled_departure_time', 'actual_departure_time',
                         'scheduled_arrival_time', 'actual_arrival_time']]
info_trips

stop_name  \
operator_name                   line_id date_id product_id                       
Schweizerische Bundesbahnen SBB 11031   0.0     Zug           Lancy-Pont-Rouge   
                                                Zug                     Genève   
                                                Zug              Les Tuileries   
                                                Zug           Genthod-Bellevue   
                                                Zug           Creux-de-Genthod   
                                                Zug                    Versoix   
                                                Zug                 Pont-Céard   
                                                Zug                       Mies   
                                                Zug                     Tannay   
                                                Zug                     Coppet   
                                        1.0     Zug           Lancy-Pont-Rouge   
                                                Zug                     Genève   
                                                Zug            Genève-Sécheron   
                                                Zug                   Chambésy   
                                                Zug              Les Tuileries   
                                                Zug           Genthod-Bellevue   
                                                Zug           Creux-de-Genthod   
                                                Zug                    Versoix   
                                                Zug                 Pont-Céard   
                                                Zug                       Mies   
                                                Zug                     Tannay   
                                                Zug                     Coppet   
                                11032   0.0     Zug                     Coppet   
                                                Zug                     Tannay   
                                                Zug                       Mies   
                                                Zug                 Pont-Céard   
                                                Zug                    Versoix   
                                                Zug           Creux-de-Genthod   
                                                Zug           Genthod-Bellevue   
                                                Zug              Les Tuileries   
...                                                                        ...   
                                12289   1.0     Zug                   Vallorbe   
                                                Zug                     Le Day   
                                                Zug               Bretonnières   
                                                Zug          Croy-Romainmôtier   
                                                Zug                      Arnex   
                                                Zug                  La Sarraz   
                                                Zug         Cossonay-Penthalaz   
                                                Zug          Vufflens-la-Ville   
                                                Zug                   Bussigny   
                                                Zug                  Renens VD   
                                                Zug              Prilly-Malley   
                                                Zug                   Lausanne   
                                                Zug                      Pully   
                                                Zug                      Lutry   
                                                Zug                Villette VD   
                                                Zug                      Cully   
                                                Zug

In [97]:
#visu_trips['date_id'] = visu_trips[['scheduled_terminus_departure_time', 'scheduled_terminus_arrival_time']].apply(date_to_integer, axis=1)
visu_trips = visu_trips.set_index(['operator_name', 'stop_list', 'date_id', 'product_id']).sort_index(axis=0, level=['stop_list'])
visu_trips = visu_trips[['line_id', 'scheduled_terminus_departure_time', 'scheduled_terminus_arrival_time', 
                         'actual_terminus_departure_time', 'actual_terminus_arrival_time']]

In [99]:
visu_trips

line_id  \
operator_name                   stop_list                                          date_id product_id           
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... 0       Zug             12   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... 0       Zug          11078   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... 0       Zug          11072   
                                                                                           Zug          11074   
                                                                                           Zug          11076   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... 0       Zug          11032   
                                                                                           Zug          11034   
                                                                                           Zug          11036   
                                                                                           Zug          11038   
                                                                                           Zug          11040   
                                                                                           Zug          11042   
                                                                                           Zug          11044   
                                                                                           Zug          11046   
                                                                                           Zug          11048   
                                                                                           Zug          11050   
                                                                                           Zug          11052   
                                                                                           Zug          11054   
                                                                                           Zug          11056   
                                                                                           Zug          11058   
                                                                                           Zug          11060   
                                                                                           Zug          11062   
                                                                                           Zug          11064   
                                                                                           Zug          11066   
                                                                                           Zug          11068   
                                                                                           Zug          11070   
                                Genève | Vernier | Meyrin | Zimeysa | Satigny |... 0       Zug          11214   
                                                                                           Zug          11216   
                                                                                           Zug          11220   
                                                                                           Zug          11222   
                                                                                           Zug          11224   
...                                                                                                       ...   
                                Vallorbe | Le Day | Bretonnières | Croy-Romainm... 0       Zug          12231   
                                                                                           Zug          12237   
                                                                                           Zug          12241   
                                  

# SCHEDULES

In [102]:
# Create table that contains scheduling information for each node in the network
# A schedule is essentially caracterized by the origin node, the destination node, the mean of 
# transport (product_id) and all the scheduled departure times from the node

# Initialize empty dataframe
df_partial_schedules = pd.DataFrame()
# Get all trips and their associated mean of transport from table 'visu_trips'
trips = set(visu_trips.index.get_values())
# Iterate on each trip to construct arcs
for _, stop_list, date_id, transport in trips:
    # list of stop names along the trip
    stop_names = stop_list.split(' | ')
    # create arcs associated with the trip
    trip_arcs = list(zip(stop_names[0:-1:1], stop_names[1::1]))
    # temporary dataframe
    temp = pd.DataFrame(trip_arcs, columns=['origin', 'destination'])
    temp['product_id'] = transport
    temp['date_id'] = date_id
    # concatenate new arcs with existing arcs in a single dataframe
    df_partial_schedules = pd.concat([df_partial_schedules, temp])

In [103]:
df_partial_schedules = df_partial_schedules.reset_index(drop=True)

In [105]:
# print the result
df_partial_schedules.head()

,origin,destination,product_id,date_id
0,Genève,Vernier,Zug,0
1,Vernier,Meyrin,Zug,0
2,Meyrin,Zimeysa,Zug,0
3,Zimeysa,Satigny,Zug,0
4,Satigny,Russin,Zug,0


In [106]:
df_schedules = pd.DataFrame()

for index, arc in df_partial_schedules.iterrows():
    origin = arc['origin']
    destination = arc['destination']
    #print(origin, destination)
    # temporary dataframe with all lines having either origin, destination or both in their stop list
    temp = info_trips[((info_trips['stop_name'] == origin) | (info_trips['stop_name'] == destination))]
    temp = temp.reset_index('line_id')
    # Get arcs
    for line_id in temp['line_id'].unique():
        # utility variable to find arcs in the dataframe 'info_trips'
        cond = temp[temp['line_id'] == line_id]
        # 1. A line must contain both stops included in the arc
        # 2. As stops are stored in the order they are visited, origin must be above destination
        if len(cond) == 2 and (cond.iloc[:1]['stop_name'] == origin).bool():
            #temp = temp.set_index(['line_id'])
            # compute scheduled arc departure time
            scheduled_departure_time = cond['scheduled_departure_time'].iloc[0]
            # compute scheduled arc arrival time
            scheduled_arrival_time = cond['scheduled_arrival_time'].iloc[1]
            # compute scheduled arc travel time
            scheduled_travel_time = scheduled_arrival_time - scheduled_departure_time
            
            # compute actual arc departure time
            actual_departure_time = cond['actual_departure_time'].iloc[0]
            # compute actual arc arrival time
            actual_arrival_time = cond['actual_arrival_time'].iloc[1]
            # compute actual arc travel time
            actual_travel_time = actual_arrival_time - actual_departure_time
            
            # Partial construction of the arc 
            to_concat = cond.iloc[:1].drop(['latitude', 'longitude', 'stop_name'], axis=1)
            to_concat['scheduled_departure_time'] = scheduled_departure_time
            to_concat['scheduled_arrival_time'] = scheduled_arrival_time
            to_concat['scheduled_travel_time'] = scheduled_travel_time
            to_concat['actual_departure_time'] = actual_departure_time
            to_concat['actual_arrival_time'] = actual_arrival_time
            to_concat['actual_travel_time'] = actual_travel_time
            to_concat['origin'] = origin
            to_concat['origin_coord'] = cond['longitude'].iloc[0] + ':' + cond['latitude'].iloc[0]
            to_concat['destination'] = destination
            to_concat['destination_coord'] = cond['longitude'].iloc[1] + ':' + cond['latitude'].iloc[1]
            to_concat = to_concat.reset_index()
            to_concat = to_concat.drop('operator_name', axis=1)
            #to_concat = to_concat.set_index(['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id'])
            to_concat = to_concat.reset_index().drop(['index'], axis=1)
            
            df_schedules = pd.concat([df_schedules, to_concat])

In [107]:
# change format to datetime
df_schedules['actual_travel_time'] = pd.to_datetime(df_schedules['actual_travel_time'], format='%H:%M:%S', errors='ignore')
# add column travel delay that helps to find parameters
#df_schedules['travel_delay [sec]'] = ((df_schedules['actual_travel_time'] - df_schedules['scheduled_travel_time']) / np.timedelta64(1, 's'))
# reorder columns
df_schedules = df_schedules[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id', 'line_id',
                     'scheduled_departure_time', 'scheduled_arrival_time', 'scheduled_travel_time', 
                     'actual_departure_time', 'actual_arrival_time', 'actual_travel_time']]
df_schedules = df_schedules.reset_index(drop=True)

In [109]:
df_schedules

,origin,origin_coord,destination,destination_coord,product_id,line_id,scheduled_departure_time,scheduled_arrival_time,scheduled_travel_time,actual_departure_time,actual_arrival_time,actual_travel_time
0,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11031,2017-09-13 09:02:00,2017-09-14 09:00:00,23:58:00,2017-09-13 09:05:13,2017-09-14 09:10:30,1 days 00:05:17
1,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11032,2017-09-13 09:28:00,2017-09-14 09:26:00,23:58:00,2017-09-13 09:31:47,2017-09-14 09:35:43,1 days 00:03:56
2,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11033,2017-09-13 09:32:00,2017-09-14 09:28:00,23:56:00,2017-09-13 09:36:12,2017-09-14 09:33:25,0 days 23:57:13
3,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11034,2017-09-13 09:58:00,2017-09-14 09:56:00,23:58:00,2017-09-13 10:02:20,2017-09-14 10:06:56,1 days 00:04:36
4,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11035,2017-09-13 10:02:00,2017-09-14 09:58:00,23:56:00,2017-09-13 10:06:39,2017-09-14 10:01:08,0 days 23:54:29
5,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11036,2017-09-13 10:28:00,2017-09-14 10:26:00,23:58:00,2017-09-13 10:29:09,2017-09-14 10:29:15,1 days 00:00:06
6,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11037,2017-09-13 10:32:00,2017-09-14 10:28:00,23:56:00,2017-09-13 10:33:09,2017-09-14 10:29:22,0 days 23:56:13
7,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11038,2017-09-13 10:58:00,2017-09-14 10:56:00,23:58:00,2017-09-13 10:58:43,2017-09-14 10:55:39,0 days 23:56:56
8,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11039,2017-09-13 11:02:00,2017-09-14 10:58:00,23:56:00,2017-09-13 11:03:10,2017-09-14 11:04:36,1 days 00:01:26
9,Genève,6.142452:46.210203,Vernier,6.142452:46.210203,Zug,11040,2017-09-13 11:28:00,2017-09-14 11:26:00,23:58:00,2017-09-13 11:32:25,2017-09-14 11:27:32,0 days 23:55:07


In [110]:
df_schedules_viz = df_schedules
df_schedules_viz = df_schedules_viz.set_index(['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id'])
df_schedules_viz = df_schedules_viz[['scheduled_departure_time', 'scheduled_arrival_time', 'scheduled_travel_time',
                'actual_departure_time', 'actual_arrival_time', 'actual_travel_time']]

In [111]:
df_schedules_viz.head()

scheduled_departure_time  \
origin origin_coord       destination destination_coord  product_id                            
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug             2017-09-13 09:02:00   
                                                         Zug             2017-09-13 09:28:00   
                                                         Zug             2017-09-13 09:32:00   
                                                         Zug             2017-09-13 09:58:00   
                                                         Zug             2017-09-13 10:02:00   

                                                                    scheduled_arrival_time  \
origin origin_coord       destination destination_coord  product_id                          
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug           2017-09-14 09:00:00   
                                                         Zug           2017-09-14 09:26:00   
                                                         Zug           2017-09-14 09:28:00   
                                                         Zug           2017-09-14 09:56:00   
                                                         Zug           2017-09-14 09:58:00   

                                                                    scheduled_travel_time  \
origin origin_coord       destination destination_coord  product_id                         
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug                     23:58:00   
                                                         Zug                     23:58:00   
                                                         Zug                     23:56:00   
                                                         Zug                     23:58:00   
                                                         Zug                     23:56:00   

                                                                    actual_departure_time  \
origin origin_coord       destination destination_coord  product_id                         
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug          2017-09-13 09:05:13   
                                                         Zug          2017-09-13 09:31:47   
                                                         Zug          2017-09-13 09:36:12   
                                                         Zug          2017-09-13 10:02:20   
                                                         Zug          2017-09-13 10:06:39   

                                                                    actual_arrival_time  \
origin origin_coord       destination destination_coord  product_id                       
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug        2017-09-14 09:10:30   
                                                         Zug        2017-09-14 09:35:43   
                                                         Zug        2017-09-14 09:33:25   
                                                         Zug        2017-09-14 10:06:56   
                                                         Zug        2017-09-14 10:01:08   

                                                                    actual_travel_time  
origin origin_coord       destination destination_coord  product_id                     
Genève 6.142452:46.210203 Vernier     6.142452:46.210203 Zug           1 days 00:05:17  
                                                         Zug           1 days 00:03:56  
                                                         Zug           0 days 23:57:13  
                                                         Zug           1 days 00:04:36  
                                                         Zug           0 days 23:54:29

# EDGES

In [112]:
# Define time slots
# WARNING those variable names should not be reassigned in the notebook
minuit = datetime.time(0, 0)
midi = datetime.time(12, 0)
defined_timeslots = [str(minuit) + '-' + str(midi), str(midi) + '-' + str(minuit)]

In [113]:
# Utility function to group edge departure times by time slot
def travel_timeslots(index):   
    timestamp = edge_travels.loc[index]['scheduled_departure_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: # midi <= target < minuit
        return defined_timeslots[1]

In [114]:
# Create dataframe with necessary edge informations to perform routing
df_edges = pd.DataFrame(columns=['fromNode', 'fromCoord', 'toNode', 'toCoord', 
                                 'product_id', 'timeslots', 'edge_params [µ, s]'])
# List of all edges
edges = df_schedules[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id']].drop_duplicates()

for index, values in edges.iterrows():
        fromNode = values.iloc[0]
        fromCoord = values.iloc[1]
        toNode = values.iloc[2]
        toCoord = values.iloc[3]
        transport = values.iloc[4]
        
        # Get all travel through the edge
        edge_travels = df_schedules[((df_schedules['origin'] == fromNode) & (df_schedules['origin_coord'] == fromCoord) &
                                    (df_schedules['destination'] == toNode) & (df_schedules['destination_coord'] == toCoord) &
                                    (df_schedules['product_id'] == transport))]
        # keep only relevant columns
        edge_travels = edge_travels[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id', 
                                     'scheduled_departure_time', 'scheduled_travel_time', 'actual_travel_time']]
        # add travel delays for the edge
        edge_travels['travel_delay [sec]'] = (edge_travels['actual_travel_time'] - edge_travels['scheduled_travel_time']) / np.timedelta64(1, 's')
        # Group edges by scheduled departure time slots
        edge_grouped = edge_travels.groupby(travel_timeslots, axis=0)['travel_delay [sec]']
        
        # timeslots with at least one element in the dataframe 'edge_travels'
        observed_timeslots = list(edge_grouped.groups.keys())
        # instantiate parameter list
        edge_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'edge_travels' associated with the timeslot 'name'
                group_indices = edge_grouped.groups[name]
                # rows in the dataframe 'edge_travels' associated with the timeslot 'name'
                group_elems = edge_travels.loc[group_indices]['travel_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                edge_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                edge_params.append((nan, nan))
        
        
        # Generate node information and add it to the dataframe df_nodes
        #print(len(timeslots), len(departure_params), len(arrival_params))
        if len(defined_timeslots) == len(edge_params):
            to_concat = pd.DataFrame({'fromNode': fromNode, 'fromCoord': fromCoord, 'toNode': toNode, 'toCoord': toCoord, 
                                      'product_id': transport, 'timeslots': defined_timeslots, 
                                      'edge_params [µ, s]': edge_params})
            df_edges = pd.concat([df_edges, to_concat], axis=0)        
        '''else:
            check.append(('j=' + str(j), 
                          'timeslots='+str(len(timeslots)), 
                          'edge_params='+str(len(departure_params))))'''

TypeError: integer argument expected, got float

In [ ]:
df_edges = df_edges[['fromNode', 'fromCoord', 'toNode', 'toCoord', 'product_id', 'timeslots', 'edge_params [µ, s]']]
df_edges = df_edges.reset_index(drop=True)
df_edges.head()

# NODES

In [ ]:
# Get list of all nodes in the network

# Origin nodes
nodes_1 = df_schedules[['origin', 'origin_coord', 'product_id']].drop_duplicates()
nodes_1.columns = ['node', 'coord', 'product_id']
# Destination nodes
nodes_2 = df_schedules[['destination', 'destination_coord', 'product_id']].drop_duplicates()
nodes_2.columns = nodes_1.columns

# Construct dataframe of nodes
nodes = pd.concat([nodes_1, nodes_2], axis=0).drop_duplicates().reset_index(drop=True)

In [ ]:
nodes.head()

In [ ]:
# Utility function to group node departures by time slot
def departure_date_range(index):   
    timestamp = from_node.loc[index]['scheduled_departure_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: # midi <= target < minuit:
        return defined_timeslots[1]

In [ ]:
# Utility function to group node arrivals by time slot
def arrival_date_range(index):
    timestamp = to_node.loc[index]['scheduled_arrival_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: #midi <= target < minuit:
        return defined_timeslots[1]

In [ ]:
# Create dataframe with necessary node information to perform routing
df_nodes = pd.DataFrame(columns=['node', 'coord', 'product_id', 'timeslots', 'departure_params [µ, s]', 'arrival_params [µ, s]'])

j = 0
for index, values in nodes.iterrows():
        node = values.iloc[0]
        coord = values.iloc[1]
        transport = values.iloc[2]    
        # Get all edges leaving the current node
        from_node = df_schedules[((df_schedules['origin'] == node) & (df_schedules['origin_coord'] == coord))]
        # Keep only relevant columns
        from_node = from_node[['origin', 'origin_coord', 'destination','destination_coord', 'product_id', 'scheduled_departure_time', 'actual_departure_time']]
        # Add departure delay values for the node 
        from_node['departure_delay [sec]'] = (from_node['actual_departure_time'] - from_node['scheduled_departure_time']) / np.timedelta64(1, 's')
        # Group leaving edges by scheduled departure time slots
        from_grouped = from_node.groupby(departure_date_range, axis=0)['departure_delay [sec]']
        # mean and standard deviation of departure delays for each time slot of the current node
        #departure_params = [(round(group.mean(), 1), round(group.std(), 1)) for _, group in from_grouped]
        
        # timeslots with at least one element in the dataframe 'from_node'
        observed_timeslots = list(from_grouped.groups.keys())
        dep_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'from_node' associated with the timeslot 'name'
                group_indices = from_grouped.groups[name]
                # rows in the dataframe 'from_node' associated with the timeslot 'name'
                group_elems = from_node.loc[group_indices]['departure_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                dep_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                dep_params.append((nan, nan))

        # Get all edges arriving at the current node
        to_node = df_schedules[((df_schedules['destination'] == node) & (df_schedules['destination_coord'] == coord))]
        # Keep only relevant columns
        to_node = to_node[['origin', 'origin_coord', 'destination','destination_coord', 'product_id',  'scheduled_arrival_time', 'actual_arrival_time']]
        # Add arrival delay values for the node
        to_node['arrival_delay [sec]'] = (to_node['actual_arrival_time'] - to_node['scheduled_arrival_time']) / np.timedelta64(1, 's')
        # Group arriving edges by scheduled arrival time
        to_grouped = to_node.groupby(arrival_date_range, axis=0)['arrival_delay [sec]']
        # mean and standard deviation of arrival delays for each time slot of the current node
        # arrival_params = [(round(group.mean(), 1), round(group.std(), 1)) for _, group in to_grouped]
        
        # timeslots with at least one element in the dataframe 'to_node'
        observed_timeslots = list(to_grouped.groups.keys())
        arr_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'from_node' associated with the timeslot 'name'
                group_indices = to_grouped.groups[name]
                # rows in the dataframe 'from_node' associated with the timeslot 'name'
                group_elems = to_node.loc[group_indices]['arrival_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                arr_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                arr_params.append((nan, nan))


        # Generate node information and add it to the dataframe df_nodes
        #print(len(timeslots), len(departure_params), len(arrival_params))
        if len(defined_timeslots) == len(dep_params) == len(arr_params):
            to_concat = pd.DataFrame({'node': node, 'coord': coord, 'product_id': transport, 'timeslot': defined_timeslots, 
                                   'departure_params [µ, s]': dep_params, 'arrival_params [µ, s]': arr_params})
            df_nodes = pd.concat([df_nodes, to_concat], axis=0)
        '''else:
            check.append(('j=' + str(j), 
                          'timeslots='+str(len(timeslots)), 
                          'departure_params='+str(len(departure_params)), 
                          'arrival_params='+str(len(arrival_params))))'''

In [ ]:
df_nodes = df_nodes[['node', 'coord', 'product_id', 'timeslot', 'departure_params [µ, s]', 'arrival_params [µ, s]']]
df_nodes = df_nodes.reset_index(drop=True)
df_nodes.head()